# The Bloq Counts Protocol

`Bloq.bloq_counts(ssa)` is a protocol that queries the number of each distinct subbloq in the parent bloq's decomposition. This can be derived from the full decomposition or annotated directly by overriding the `bloq_counts` method.

In [ ]:
from cirq_qubitization.quantum_graph.bloq_counts import get_bloq_counts_graph, print_counts_graph, \
    GraphvizCounts, markdown_counts_graph, markdown_counts_sigma, SympySymbolAllocator

## Basic counts graph of `And`

In [ ]:
from cirq_qubitization.bloq_algos.and_bloq import MultiAnd, And

graph, sigma = get_bloq_counts_graph(MultiAnd(cvs=(1,)*6))
print_counts_graph(graph)

In [ ]:
markdown_counts_graph(graph)

In [ ]:
GraphvizCounts(graph).get_svg()

In [ ]:
markdown_counts_sigma(sigma)

## Importance of bloq generalization

Often, bloqs have attributes that are important for describing their specific action but don't affect the bloq's resource cost. For example, `Rx(0.12)` and `Rx(0.13)` should probably be considered equal when counting the number of operations. Another example is given below where we group all two-bit `And` operations no matter their control values. 

`get_bloq_counts_graph` takes an optional callable that takes specific bloqs to general bloqs. See below for an example of what happens to the `MultiAnd` counts graph without generalization, and how we can use it to replace specific control values with sympy symbols.

In [ ]:
graph, sigma = get_bloq_counts_graph(MultiAnd(cvs=(1,0)*3))
GraphvizCounts(graph).get_svg()

In [ ]:
import attrs

ssa = SympySymbolAllocator()
cv1 = ssa.new_symbol('cv1')
cv2 = ssa.new_symbol('cv2')


def generalize(bloq):
    if isinstance(bloq, And):
        return attrs.evolve(bloq, cv1=cv1, cv2=cv2)
    
    return bloq

graph, sigma = get_bloq_counts_graph(MultiAnd(cvs=(1,0)*3), generalize, ssa)
GraphvizCounts(graph).get_svg()

In [ ]:
markdown_counts_sigma(sigma)